In [ ]:
#!/usr/bin/env python3.9 -m nuitka
# -*- coding: utf-8 -*-
import browser_cookie3
import requests
import configparser
import json
import os
import sys
import time
from playsound import playsound
from datetime import datetime
import telepot
import unicodedata
import urllib3
import re
import platform

search_time = 0.2  # 잔여백신을 해당 시간마다 한번씩 검색합니다. 단위: 초
urllib3.disable_warnings()

# 아래의 `load_cookie()` 에서 쿠키를 불러옴.
jar = None

# 기존 입력 값 로딩
def load_config():
    config_parser = configparser.ConfigParser()
    if os.path.exists('config.ini'):
        try:
            config_parser.read('config.ini')

            while True:
#                 skip_input = str.lower(input("기존에 입력한 정보로 재검색하시겠습니까? Y/N : "))
                skip_input = "n"
                if skip_input == "y":
                    skip_input = True
                    break
                elif skip_input == "n":
                    skip_input = False
                    break
                else:
                    print("Y 또는 N을 입력해 주세요.")

            if skip_input:
                # 설정 파일이 있으면 최근 로그인 정보 로딩
                configuration = config_parser['config']
                previous_used_type = configuration["VAC"]
                previous_top_x = configuration["topX"]
                previous_top_y = configuration["topY"]
                previous_bottom_x = configuration["botX"]
                previous_bottom_y = configuration["botY"]
                previous_only_left = configuration["onlyLeft"] == "True"
                return previous_used_type, previous_top_x, previous_top_y, previous_bottom_x, previous_bottom_y, previous_only_left
            else:
                return None, None, None, None, None, None
        except ValueError:
            return None, None, None, None, None, None
    return None, None, None, None, None, None

# cookie.ini 안의 [chrome][cookie_file] 에서 경로를 로드함.
def load_cookie_config():
    global jar

    config_parser = configparser.ConfigParser(interpolation=None)
    if os.path.exists('cookie.ini'):
        config_parser.read('cookie.ini')
        try:
            cookie_file = config_parser.get(
                'chrome', 'cookie_file', fallback=None)
            if cookie_file is None:
                return None

            indicator = cookie_file[0]
            if indicator == '~':
                cookie_path = os.path.expanduser(cookie_file)
            elif indicator in ('%', '$'):
                cookie_path = os.path.expandvars(cookie_file)
            else:
                cookie_path = cookie_file

            cookie_path = os.path.abspath(cookie_path)

            if os.path.exists(cookie_path):
                return cookie_path
            else:
                print("지정된 경로에 쿠키 파일이 존재하지 않습니다. 기본값으로 시도합니다.")
                return None
        except Exception:  # 정확한 오류를 몰라서 전부 Exception
            return None
    return None

def load_saved_cookie() -> bool:
    #  print('saved cookie loading')
    config_parser = configparser.ConfigParser(interpolation=None)

    global jar

    if os.path.exists('cookie.ini'):
        try:
            config_parser.read('cookie.ini')
            cookie = config_parser['cookie_values']['_kawlt'].strip()

            if cookie is None or cookie == '':
                return False

            jar = {'_kawlt': cookie}
            return True
        except Exception:
            return False

    return False

def dump_cookie(value):
    config_parser = configparser.ConfigParser()
    config_parser.read('cookie.ini')
    
    with open('cookie.ini', 'w') as cookie_file:
        config_parser['cookie_values'] = {
            '_kawlt': value
        }
        config_parser.write(cookie_file)

# cookie 경로가 입력되지 않았을시, 쿠키 파일이 Default 경로에 있는지 확인함
# 경로가 입력되었거나, Default 경로의 쿠키가 존재해야 global jar 함수에 cookie를 로드함.
def load_cookie_from_chrome() -> None:
    global jar

    cookie_file = load_cookie_config()
    if cookie_file is False:
        return

    if cookie_file is None:
        cookie_path = None
        os_type = platform.system()
        if os_type == "Linux":
            # browser_cookie3 also checks beta version of google chrome's cookie file.
            cookie_path = os.path.expanduser(
                "~/.config/google-chrome/Default/Cookies")
            if os.path.exists(cookie_path) is False:
                cookie_path = os.path.expanduser(
                    "~/.config/google-chrome-beta/Default/Cookies")
        elif os_type == "Darwin":
            cookie_path = os.path.expanduser(
                "~/Library/Application Support/Google/Chrome/Default/Cookies")
        elif os_type == "Windows":
            cookie_path = os.path.expandvars(
                "%LOCALAPPDATA%/Google/Chrome/User Data/Default/Cookies")
        else:  # Jython?
            print("지원하지 않는 환경입니다.")
            close()

        if os.path.exists(cookie_path) is False:
            print("기본 쿠키 파일 경로에 파일이 존재하지 않습니다. 아래 링크를 참조하여 쿠키 파일 경로를 지정해주세요.\n" +
                  "https://github.com/SJang1/korea-covid-19-remaining-vaccine-macro/discussions/403")
            close()

    jar = browser_cookie3.chrome(
        cookie_file=cookie_file, domain_name=".kakao.com")

    # 쿠키를 cookie.ini에 저장한다
    for cookie in jar:
        dump_cookie(cookie.value)

def load_search_time():
    global search_time

    config_parser = configparser.ConfigParser()
    if os.path.exists('config.ini'):
        config_parser.read('config.ini')
        input_time = config_parser.getfloat(
            'config', 'search_time', fallback=0.2)

        if input_time < 0.1:
            search_time = 0.1
        else:
            search_time = input_time

def check_user_info_loaded():
    global jar
#     print(jar)
    user_info_api = 'https://vaccine.kakao.com/api/v1/user'
    user_info_response = requests.get(
        user_info_api, headers=Headers.headers_vacc, cookies=jar, verify=False)
    user_info_json = json.loads(user_info_response.text)
    if user_info_json.get('error'):
        print("API에러.. 쿠키 다시 체크중")
        # cookie.ini에 있는 쿠키가 유통기한 지났을 수 있다
        # 비교 위해서 cookie.ini 쿠키를 'prev_jar'에 저장한다
        prev_jar = jar 
        load_cookie_from_chrome()

        # 크롬 브라우저에서 새로운 쿠키를 찾았으면 다시 체크 시작 한다
        if prev_jar != jar:
            #  print('new cookie value from chrome detected')
            check_user_info_loaded()
            return

        print("사용자 정보를 불러오는데 실패하였습니다.")
        print("Chrome 브라우저에서 카카오에 제대로 로그인되어있는지 확인해주세요.")
        print("로그인이 되어 있는데도 안된다면, 카카오톡에 들어가서 잔여백신 알림 신청을 한번 해보세요. 정보제공 동의가 나온다면 동의 후 다시 시도해주세요.")
        close()
    else:
        user_info = user_info_json.get("user")
        for key in user_info:
            value = user_info[key]
            # print(key, value)
            if key != 'status':
                continue
            if key == 'status' and value == "NORMAL":
                print("사용자 정보를 불러오는데 성공했습니다.")
                break
            elif key == 'status' and value == "UNKNOWN":
                print("상태를 알 수 없는 사용자입니다. 1339 또는 보건소에 문의해주세요.")
                close()
            else:
                print("이미 접종이 완료되었거나 예약이 완료된 사용자입니다.")
                close(success=None)

def fill_str_with_space(input_s, max_size=40, fill_char=" "):
    """
    - 길이가 긴 문자는 2칸으로 체크하고, 짧으면 1칸으로 체크함.
    - 최대 길이(max_size)는 40이며, input_s의 실제 길이가 이보다 짧으면
    남은 문자를 fill_char로 채운다.
    """
    length = 0
    for c in input_s:
        if unicodedata.east_asian_width(c) in ["F", "W"]:
            length += 2
        else:
            length += 1
    return input_s + fill_char * (max_size - length)

# Something is wrong here
def is_in_range(coord_type, coord, user_min_x=-180.0, user_max_y=90.0):
    korea_coordinate = {  # Republic of Korea coordinate
        "min_x": 124.5,
        "max_x": 132.0,
        "min_y": 33.0,
        "max_y": 38.9
    }
    try:
        if coord_type == "x":
            return max(korea_coordinate["min_x"], user_min_x) <= float(coord) <= korea_coordinate["max_x"]
        elif coord_type == "y":
            return korea_coordinate["min_y"] <= float(coord) <= min(korea_coordinate["max_y"], user_max_y)
        else:
            return False
    except ValueError:
        # float 이외 값 입력 방지
        return False

# pylint: disable=too-many-branches
def input_config():
    global vaccine_type
    vaccine_candidates = [
        {"name": "아무거나", "code": "ANY"},
        {"name": "화이자", "code": "VEN00013"},
        {"name": "모더나", "code": "VEN00014"},
        {"name": "아스트라제네카", "code": "VEN00015"},
        {"name": "얀센", "code": "VEN00016"},
        {"name": "(미사용)", "code": "VEN00017"},
        {"name": "(미사용)", "code": "VEN00018"},
        {"name": "(미사용)", "code": "VEN00019"},
        {"name": "(미사용)", "code": "VEN00020"},
    ]

    vaccine_type = vaccine_type
    top_x = "129.4369078955544"
    top_y = "35.931968680267204"
    bottom_x = "129.28111700916696"
    bottom_y = "36.059338527137314"
    only_left = "true"
    dump_config(vaccine_type, top_x, top_y, bottom_x, bottom_y, only_left)
    return vaccine_type, top_x, top_y, bottom_x, bottom_y, only_left

# pylint: disable=too-many-arguments
def dump_config(vaccine_type, top_x, top_y, bottom_x, bottom_y, only_left):
    config_parser = configparser.ConfigParser()
    config_parser['config'] = {}
    conf = config_parser['config']
    conf['VAC'] = vaccine_type
    conf["topX"] = top_x
    conf["topY"] = top_y
    conf["botX"] = bottom_x
    conf["botY"] = bottom_y
    conf["search_time"] = str(search_time)
    conf["onlyLeft"] = only_left

    with open("config.ini", "w") as config_file:
        config_parser.write(config_file)

def clear():
    if 'win' in sys.platform.lower():
        os.system('cls')
    else:
        os.system('clear')

def close(success=False):
    if success is True:
        print("잔여백신 예약 성공!! \n 카카오톡지갑을 확인하세요.")
    elif success is False:
        print("오류와 함께 잔여백신 예약 프로그램이 종료되었습니다.")
    else:
        pass
    input("Press Enter to close...")
    sys.exit()

def pretty_print(json_object):
    status = "잔여백신 없음"
    for org in json_object["organizations"]:
        if org.get('status') == "CLOSED" or org.get('status') == "EXHAUSTED" or org.get('status') == "UNAVAILABLE":
            continue
        status = "감지완료"
        print(
            f"잔여갯수: {org.get('leftCounts')}\t상태: {org.get('status')}\t기관명: {org.get('orgName')}\t주소: {org.get('address')}")
    return status

class Headers:
    headers_map = {
        "Accept": "application/json, text/plain, */*",
        "Content-Type": "application/json;charset=utf-8",
        "Origin": "https://vaccine-map.kakao.com",
        "Accept-Language": "en-us",
        "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 14_7 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148 KAKAOTALK 9.4.2",
        "Referer": "https://vaccine-map.kakao.com/",
        "Accept-Encoding": "gzip, deflate",
        "Connection": "Keep-Alive",
        "Keep-Alive": "timeout=5, max=1000"
    }
    headers_vacc = {
        "Accept": "application/json, text/plain, */*",
        "Content-Type": "application/json;charset=utf-8",
        "Origin": "https://vaccine.kakao.com",
        "Accept-Language": "en-us",
        "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 14_7 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148 KAKAOTALK 9.4.2",
        "Referer": "https://vaccine.kakao.com/",
        "Accept-Encoding": "gzip, deflate",
        "Connection": "Keep-Alive",
        "Keep-Alive": "timeout=5, max=1000"
    }

def try_reservation(organization_code_success, vaccine_type):
    reservation_url = 'https://vaccine.kakao.com/api/v2/reservation'
    data = {"from": "Map", "vaccineCode": vaccine_type,
            "orgCode": organization_code, "distance": None}
    response = requests.post(reservation_url, data=json.dumps(
        data), headers=Headers.headers_vacc, cookies=jar, verify=False)    
    response_json = json.loads(response.text)
    for key in response_json:
        value = response_json[key]
        if key != 'code':
            continue
        if key == 'code' and value == "NO_VACANCY":
            print("잔여백신 접종 신청이 선착순 마감되었습니다.")
            time.sleep(0.08)
        elif key == 'code' and value == "TIMEOUT":
            print("TIMEOUT, 예약을 재시도합니다.")
            retry_reservation(organization_code, vaccine_type)
        elif key == 'code' and value == "SUCCESS":
            print("백신접종신청 성공!!!")
            organization_code_success = response_json.get("organization")
            print(
                f"병원이름: {organization_code_success.get('orgName')}\t" +
                f"전화번호: {organization_code_success.get('phoneNumber')}\t" +
                f"주소: {organization_code_success.get('address')}")
            close(success=True)
        else:
            print("ERROR. 아래 메시지를 보고, 예약이 신청된 병원 또는 1339에 예약이 되었는지 확인해보세요.")
            print(response.text)
            close()
    f = open("response.txt", "a") # 파일 열기
    print(response, file=f) # 파일 저장하기
    f.close()

def retry_reservation(organization_code, vaccine_type):
    reservation_url = 'https://vaccine.kakao.com/api/v1/reservation/retry'

    data = {"from": "Map", "vaccineCode": vaccine_type,
            "orgCode": organization_code, "distance": None}
    response = requests.post(reservation_url, data=json.dumps(
        data), headers=Headers.headers_vacc, cookies=jar, verify=False)
    response_json = json.loads(response.text)
    for key in response_json:
        value = response_json[key]
        if key != 'code':
            continue
        if key == 'code' and value == "NO_VACANCY":
            print("잔여백신 접종 신청이 선착순 마감되었습니다.")
            time.sleep(0.08)
        elif key == 'code' and value == "SUCCESS":
            print("백신접종신청 성공!!!")
            organization_code_success = response_json.get("organization")
            print(
                f"병원이름: {organization_code_success.get('orgName')}\t" +
                f"전화번호: {organization_code_success.get('phoneNumber')}\t" +
                f"주소: {organization_code_success.get('address')}")
            close(success=True)
        else:
            print("ERROR. 아래 메시지를 보고, 예약이 신청된 병원 또는 1339에 예약이 되었는지 확인해보세요.")
            print(response.text)
            close()

def find_vaccine(vaccine_type, top_x, top_y, bottom_x, bottom_y, only_left):
    global jar, reserve_run
    url = 'https://vaccine-map.kakao.com/api/v3/vaccine/left_count_by_coords'
    data = {"bottomRight": {"x": bottom_x, "y": bottom_y}, "onlyLeft": "true", "order": "latitude",
            "topLeft": {"x": top_x, "y": top_y}}
    done = False
    found = None
    
    try_count = 0
    while not done:
        try_count += 1
        try:
            time.sleep(search_time)
            response = requests.post(url, data=json.dumps(
                data), headers=Headers.headers_map, cookies=jar, verify=False, timeout=5)

            try:
                json_data = json.loads(response.text)
#                 if json_data["error"]:
#                     print(url, data, Headers.headers_map, jar, json_data)
                status = pretty_print(json_data)
                print("{}번째 시도중...{}".format(try_count, status), end="\r")

                for x in json_data.get("organizations"):
                    if x.get('status') == "AVAILABLE" or x.get('leftCounts') != 0:
                        found = x
                        done = True
                        break

            except json.decoder.JSONDecodeError as decodeerror:
                print("JSONDecodeError : ", decodeerror)
                print("JSON string : ", response.text)
                close()

        except requests.exceptions.Timeout as timeouterror:
            print("Timeout Error : ", timeouterror)

        except requests.exceptions.SSLError as sslerror:
            print("SSL Error : ", sslerror)
            close()

        except requests.exceptions.ConnectionError as connectionerror:
            print("Connection Error : ", connectionerror)
            # See psf/requests#5430 to know why this is necessary.
            if not re.search('Read timed out', str(connectionerror), re.IGNORECASE):
                close()

        except requests.exceptions.HTTPError as httperror:
            print("Http Error : ", httperror)
            close()

        except requests.exceptions.RequestException as error:
            print("AnyException : ", error)
            close()

    if found is None:
        find_vaccine(vaccine_type, top_x, top_y, bottom_x, bottom_y, only_left)
        return None

    print(f"{found.get('orgName')} 에서 백신을 {found.get('leftCounts')}개 발견했습니다.")
    print(f"주소는 : {found.get('address')} 입니다.")
    organization_code = found.get('orgCode')

    # 실제 백신 남은수량 확인
    vaccine_found_code = None

    if vaccine_type == "ANY":  # ANY 백신 선택
        check_organization_url = f'https://vaccine.kakao.com/api/v3/org/org_code/{organization_code}'
        check_organization_response = requests.get(check_organization_url, headers=Headers.headers_vacc, cookies=jar,
                                                   verify=False)
        check_organization_data = json.loads(
            check_organization_response.text).get("lefts")
        for x in check_organization_data:
            if x.get('leftCount') != 0:
                found = x
                print(f"{x.get('vaccineName')} 백신을 {x.get('leftCount')}개 발견했습니다.")
                vaccine_found_code = x.get('vaccineCode')
                break
            else:
                print(f"{x.get('vaccineName')} 백신이 없습니다.")

    else:
        vaccine_found_code = vaccine_type
        print(f"{vaccine_found_code} 으로 예약을 시도합니다.")

    if vaccine_found_code and reserve_run and try_reservation(organization_code, vaccine_found_code):
        return None
    else:
        find_vaccine(vaccine_type, top_x, top_y, bottom_x, bottom_y, only_left)
        return None

def main_function():
    got_cookie = load_saved_cookie()
    if got_cookie is False:
        load_cookie_from_chrome()

    load_search_time()
    check_user_info_loaded()
    previous_used_type, previous_top_x, previous_top_y, previous_bottom_x, previous_bottom_y, only_left = load_config()
    if previous_used_type is None:
        vaccine_type, top_x, top_y, bottom_x, bottom_y, only_left = input_config()
    else:
        vaccine_type, top_x, top_y, bottom_x, bottom_y = previous_used_type, previous_top_x, previous_top_y, previous_bottom_x, previous_bottom_y
    find_vaccine(vaccine_type, top_x, top_y, bottom_x, bottom_y, only_left)
    close()

# ===================================== run ===================================== #
vaccine_type = "ANY"
reserve_run = False
if __name__ == '__main__':
    main_function()

API에러.. 쿠키 다시 체크중
사용자 정보를 불러오는데 성공했습니다.
